In [3]:
# -*- coding:utf-8 -*-
import argparse
import warnings

warnings.filterwarnings('ignore')

import pickle, random, sys

sys.path.append('D:\\zhaixu\\Thesis_Code')

import os, pickle, random, sys

import numpy as np
import copy
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.regularizers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import model_from_json

from sklearn.model_selection import train_test_split

from mltools import build_model
from mltools import get_seq_data

# from rmlmodel.Sequence.CLDNN import CLDNNLikeModel
# from rmlmodel.Sequence.ResNet import ResNetLikeModel
# from rmlmodel.Sequence.VGG import VGGLikeModel

from rmlmodel.Sequence.vtcnn2 import VTCNN2
from rmlmodel.Sequence.CNN2 import CNN2
from rmlmodel.Sequence.CNN2Model import CNN2Model

from rmlmodel.Sequence.CLDNNLikeModel import CLDNNLikeModel
from rmlmodel.Sequence.CLDNNLikeModel1 import CLDNNLikeModel1
from rmlmodel.Sequence.CLDNNLikeModel2 import CLDNNLikeModel2

from rmlmodel.Sequence.CGDNN import CGDNN
from rmlmodel.Sequence.CuDNNLSTMModel import LSTMModel
from rmlmodel.Sequence.DAE import DAE
from rmlmodel.Sequence.DCNNPF import DCNNPF
from rmlmodel.Sequence.DenseNet import DenseNet
from rmlmodel.Sequence.GRUModel import GRUModel
from rmlmodel.Sequence.ICAMC import ICAMC
from rmlmodel.Sequence.MCLDNN import MCLDNN
from rmlmodel.Sequence.MCNET import MCNET
from rmlmodel.Sequence.PETCGDNN import PETCGDNN
from rmlmodel.Sequence.ResNet import ResNet

from trigger_config import load_data
from trigger_config import set_trigger_config

from plot_tools import plot_signal
from plot_tools import plot_constellation

In [2]:
X_train,X_test,Y_train,Y_test,mods,lbl,snrs,train_idx,test_idx = load_data()

筛选后的数据集总数： 110000
信噪比范围: 0 到 18
调制方式 11 种: ['8PSK', 'AM-DSB', 'AM-SSB', 'BPSK', 'CPFSK', 'GFSK', 'PAM4', 'QAM16', 'QAM64', 'QPSK', 'WBFM']


In [4]:
def eval(model,X_test,Y_test):
    # model0:原模型测试
    classes = mods
    acc = {}
    for snr in snrs:

        # extract classes @ SNR
        test_SNRs = list(map(lambda x: lbl[x][1], test_idx))

        # CA
    #     test_X_i = X_test_benign.values[np.where(np.array(test_SNRs)==snr)]
    #     test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   

        #ASR
        test_X_i = X_test.values[np.where(np.array(test_SNRs)==snr)]
        test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   

        test_Y_i_hat = model.predict(test_X_i)
        # estimate classes
        test_Y_i_hat = model.predict(test_X_i)
        conf = np.zeros([len(classes),len(classes)])
        confnorm = np.zeros([len(classes),len(classes)])
        for i in range(0,test_X_i.shape[0]):
            j = list(test_Y_i[i,:]).index(1)
            k = int(np.argmax(test_Y_i_hat[i,:]))
            conf[j,k] = conf[j,k] + 1
        for i in range(0,len(classes)):
            confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
        #plt.figure()
        #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))

        cor = np.sum(np.diag(conf))
        ncor = np.sum(conf) - cor
        #print("Overall Accuracy: ", cor / (cor+ncor))
        print(cor / (cor+ncor))
        acc[snr] = 1.0*cor/(cor+ncor)

In [5]:
X_train_modified, Y_train_modified = set_trigger_config(X_train.copy(), Y_train.copy(), pos_rate=0.1,
                                                        trigger_type='badnet', data_type='train')

# REP = IQ/AP/FFT
X_train_modified = get_seq_data(X_train_modified, seq_dtype ='IQ')

Poisoned training data.


In [6]:
X_test_modified, Y_test_modified = set_trigger_config(X_test.copy(), Y_test.copy(), pos_rate=0.1,
                                                        trigger_type='badnet', data_type='test')

# REP = IQ/AP/FFT
X_test_modified = get_seq_data(X_test_modified, seq_dtype ='IQ')

Evaluate ASR on poisoned test data.


In [7]:
X_test_benign = X_test.copy()

In [41]:
from utlits import form_features_time
X_train_feature_modified = form_features_time(X_train_modified[:,:,:])
X_test_feature_modified = form_features_time(X_test_modified[:,:,:])
X_test_feature_benign = form_features_time(X_test_benign[:,:,:])
print("X_train_feature_modified", X_train_feature_modified.shape)
print("X_test_feature_modified", X_test_feature_modified.shape)
print("X_test_feature_benign", X_test_feature_benign.shape)

X_I (99000, 5, 128)
X_Q (99000, 5, 128)
X_complex (99000, 5, 128)


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:36<00:00,  4.05s/it]


X_I (11000, 5, 128)
X_Q (11000, 5, 128)
X_complex (11000, 5, 128)


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  2.21it/s]


X_I (11000, 5, 128)
X_Q (11000, 5, 128)
X_complex (11000, 5, 128)


100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:04<00:00,  2.20it/s]

X_train_feature_modified (99000, 405)
X_test_feature_modified (11000, 405)
X_test_feature_benign (11000, 405)


In [42]:
from utlits import standardize_features
X_train_modified, X_test_modified = standardize_features(X_train_feature_modified,X_test_feature_modified)
X_train_modified, X_test_benign = standardize_features(X_train_feature_modified,X_test_feature_benign)

(110000, 400)
X_train_std_all, (99000, 400)
X_test_std, (11000, 400)
(110000, 400)
X_train_std_all, (99000, 400)
X_test_std, (11000, 400)


In [95]:
# devide train and val data
from sklearn.model_selection import train_test_split
X_train_std, X_val_std, y_train, y_val = train_test_split(X_train_modified.copy(), Y_train.copy(), test_size=0.00001, random_state=42)
print("X_train_std,", X_train_std.shape)
print("X_val_std,", X_val_std.shape)
print("X_test_modified,", X_test_modified.shape)

X_train_std, (98999, 400)
X_val_std, (1, 400)
X_test_modified, (11000, 400)


In [96]:
import pandas as pd
X_train_std = pd.DataFrame(X_train_std)
X_val_std = pd.DataFrame(X_val_std)

X_train_modified = pd.DataFrame(X_train_modified)
X_test_modified = pd.DataFrame(X_test_modified)
X_test_benign = pd.DataFrame(X_test_benign)

In [66]:
# inf,nan数据填充
X_train_std = (X_train_std.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 
X_val_std = (X_val_std.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 

X_train_modified = (X_train_modified.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 
X_test_modified = (X_test_modified.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 
X_test_benign = (X_test_benign.replace([np.inf, -np.inf], np.nan)).fillna(value = 0) 

In [67]:
from sklearn import datasets,model_selection,metrics,tree,preprocessing
#x=preprocessing.StandardScaler().fit_transform(x)
# 划分训练集、测试集
#x_train,x_test,y_train,y_test=model_selection.train_test_split(X_train_std,y_train,test_size=0.3)
# 导入决策树
model=tree.DecisionTreeClassifier(max_depth=35)
# 模型训练
model.fit(X_train_std.values,y_train)
#model.fit(X_train_modified.values,Y_train_modified)

DecisionTreeClassifier(max_depth=35)

In [68]:
# CA
print('CA')
y_pred=model.predict(X_test_benign.values)
# 模型性能评价
print(metrics.accuracy_score(y_pred,Y_test))

# ASR
print('ASR')
y_pred=model.predict(X_test_modified.values)
# 模型性能评价
print(metrics.accuracy_score(y_pred,Y_test_modified))

CA
0.8522727272727273
ASR
0.09245454545454546


In [105]:
# LGBMClassifier
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier

clf_multilabel = OneVsRestClassifier(LGBMClassifier(max_depth=10))
clf_multilabel.fit(X_train_modified.values,Y_train_modified)

OneVsRestClassifier(estimator=LGBMClassifier(max_depth=10))

In [106]:
from sklearn import datasets,model_selection,metrics,tree,preprocessing
y_pred=clf_multilabel.predict(X_val_std)
# 模型性能评价
print(metrics.accuracy_score(y_pred,y_val))

1.0


In [115]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier
import numpy as np
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn
import warnings
from numpy import loadtxt
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import  pyplot
from sklearn.metrics import roc_curve, auc
import matplotlib as mpl  
import matplotlib.pyplot as plt


model_xgb = xgb.XGBClassifier(max_depth=20,tree_method='gpu_hist', gpu_id=0)

clf_multilabel = OneVsRestClassifier(model_xgb)

#clf_multilabel.fit(X_train_std.values,y_train)

clf_multilabel.fit(X_train_modified.values,Y_train_modified)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=0, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=20,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, reg_alpha=None,
                                           

In [117]:
y_pred=clf_multilabel.predict(X_test_benign.values)
# 模型性能评价
print(metrics.accuracy_score(y_pred,Y_test))

0.8732727272727273


In [118]:
# model0:原模型测试
classes = mods
model = clf_multilabel
acc = {}
for snr in snrs:

    # extract classes @ SNR
    test_SNRs = list(map(lambda x: lbl[x][1], test_idx))
    test_X_i = X_test_modified.values[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test_modified[np.where(np.array(test_SNRs)==snr)]   
    
#     test_X_i = X_test_benign.values[np.where(np.array(test_SNRs)==snr)]
#     test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]   

    test_Y_i_hat = clf_multilabel.predict(test_X_i)
    # estimate classes
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(classes)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    #plt.figure()
    #plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    #print("Overall Accuracy: ", cor / (cor+ncor))
    print(cor / (cor+ncor))
    acc[snr] = 1.0*cor/(cor+ncor)

0.9132507149666349
0.9157303370786517
0.8995594713656387
0.9219467401285583
0.9212121212121213
0.9437386569872959
0.9449954914337241
0.9460201280878316
0.9649122807017544
0.9588182632050134
